<a href="https://colab.research.google.com/github/Qajirr/Qajirr/blob/main/Tarea_1_EL4114_2023_vf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tarea #1:** Optimización centrales de carga vehículos eléctricos
## **EL4114 - Optimización**
### **Primavera 2023**
##### **Profesora:** Ángela Flores Q.
##### **Auxiliar:** Felipe Keim M.
##### **Ayudantes:** Byron Castro R., Diego Gonzales D., Luis Ramírez C.
##### **Alumnos:** Vicente Pinochet R., Pablo González A



#### **Recomendaciones**

* Resuelva la tarea en su completitud en el entorno de Google Colab. Los datos del problema se cargarán automáticamente si utiliza esta plataforma.

* En cada pregunta se incluyen bloques en blanco para programar sus soluciones. Esta es solo una recomendación de estructura. Usted puede implementar su código como le sea conveniente, siempre que este sea ejecutable y entendible.

#### **Instrucciones**

* Realice una copia de este notebook en su cuenta de Google y resuelva su tarea allí. Para realizarla, siga la ruta "Archivo $\rightarrow$ Guardar una copia en Drive"

*   La tarea se puede resolver en parejas.
*   El código escrito debe ser únicamente de su autoría. Cualquier fuente externa utilizada debe ser citada claramente en las referencias al final de este Notebook. Aún así, el uso de sitios de consulta como *StackOverflow*, *TowardsDataScience*, etc. está permitido manteniendo en un nivel prudente la cantidad de código que utiliza proveniente de ellos.
*   Cualquier plagio entre códigos de las soluciones enviadas por los estudiantes será penalizado y la situación revisada por el cuerpo docente del curso.
*   Se recomienda fuertemente comentar el código para un mejor entendimiento en la corrección.
*   Verifique antes de realizar su última entrega que el código efectivamente se puede ejecutar. Si algún bloque de código o el código completo no se ejecuta, no lo entregue ya que todo bloque que no se ejecute sin errores no será corregido.
* Sea claro en sus resultados y comentarios. Al graficar no olvide incluir los nombres de los ejes, título, leyendas, etc.
* Los archivos con datos entregados por el equipo docente **no deben ser modificados**. Las pruebas del código serán realizadas con los archivos originales.
* Antes de resolver la tarea, es altamente recomendable que revise las clases auxiliares extra realizadas ya que le serán de utilidad para implementar el código con mayor facilidad.

#### **Formato de entrega**

*  Para que su entrega sea corregida, entregue a través de U-Cursos el archivo .ipynb que puede descargar de Google Colab en "Archivo $\rightarrow$ Descargar $\rightarrow$ Descargar .ipynb"

* No se aceptarán enlaces a Google Colab.

* No es necesario que redacte un informe aparte. Todas las respuestas, comentarios, gráficos, ecuaciones, etc. deben ser incluidas en este documento

#### Importe de librerías

In [ ]:
#Librerías de utilidad.
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from IPython.display import clear_output

# Instalar Pyomo y solvers.
!pip install pyomo -q
!pip install -q gurobipy
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.common.timing import TicTocTimer
from gurobipy import *

#Credenciales Google para acceder a base de datos.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
clear_output()

print('**** Pyomo instalado e importado****')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 37.7 MB/s eta 0:00:00


In [ ]:
!pip install cplex

# Objetivo de la tarea:
El propósito principal de esta tarea es fortalecer sus competencias de modelamiento de problemas de optimización, aplicando las técnicas adquiridas en clase en un contexto práctico. Además, esta tarea le brindará la oportunidad de familiarizarse con lalibrería de modelado matemático Pyomo.

Una vez finalizada esta tarea, usted estará en condiciones de:

* Formular modelos de optimización para abordar desafíos de ingeniería.
* Implementar modelos de optimización en Python mediante la utilización de la librería Pyomo.
* Resolver  modelos de optimización con el apoyo de herramientas computacionales.

#Encargo:
Esta tarea consiste de tres partes. En la primera, se le pide formular un modelo de optimización para dar solución al problema que se le presenta a continuación. En la segunda parte, se le pide implementar el problema de optimización ya formulado a un caso de estudio pequeño. Finalmente, en la tercera parte, se aplica el modelo desarrollado a un caso de estudio de mayor tamaño.  



# Contexto y problema a solucionar

La electrificación del transporte se presenta como una solución atractiva para abordar la creciente preocupación por la contaminación ambiental y la reducción de la dependencia de combustibles fósiles. Este enfoque cobra especial relevancia gracias a la integración de fuentes de energía renovable y la disminución de los costos de producción de baterías, lo que ha hecho que los vehículos eléctricos sean cada vez más competitivos en el mercado. Esto es particularmente cierto en el caso de flotas de taxis y colectivos.

En este contexto, una nueva empresa de colectivos eléctricos llamada Opticars tiene planes de iniciar sus operaciones en Santiago. La empresa ya cuenta con una flota de $K$ colectivos eléctricos, y cada uno de estos colectivos tiene la capacidad de realizar $J$ recorridos diferentes. Sin embargo, para poder dar inicio a sus operaciones de manera efectiva, Opticars necesita arrendar un equipo de carga que pueda satisfacer las necesidades de su flota. Para tomar esta decisión, la empresa puede elegir entre $R$ opciones de equipos de carga, cada una con características técnicas y costos de alquiler diferentes.

Como parte del Departamento de Planificación, su responsabilidad consiste en plantear e implementar un problema de optimización que permita a Opticars coordinar la operación de sus colectivos y seleccionar el equipo de carga más adecuado para arrendar de modo de máximizar sus utilidades.

A continuación, se proporciona una guía completa con toda la información necesaria para plantear el problema de optimización



## Guía de modelación:
La siguiente guía le será una ayuda para definir las restricciones, función objetivo, variables y conjuntos del problema.

**Problema a solucionar**
  > Se desea coordinar la operación del servicio de colectivos eléctricos durante un día y definir el equipo de carga necesario para cargar las baterías de los vehículos en ese periodo de tiempo, de modo de maximizar las utilidades de la empresa.

  > La empresa dispone de $K$ colectivos eléctricos, los que pueden realizar $J$ recorridos diferentes.  Para coordinar su operación, se debe decidir  cuáles recorridos realizará cada auto durante el día de operación, así como definir cuándo se encontrará estacionado en la estación de carga. Para poder determinar su operación, se debe tener en cuenta que los vehículos empiezan a operar a las 5:00 y terminan su operación a las 23:00. Además, se debe tener en cuenta la duración de los recorridos, el consumo energético requerido para realizarlos, y la capacidad de almacenamiento de energía de la batería, entre otros aspectos que se detallarán en los puntos siguientes.
  
  > Además, para poder dar inicio a las operaciones de manera, la empresa necesita arrendar un equipo de carga que pueda satisfacer las necesidades de su flota. Para tomar esta decisión, la empresa puede elegir un equipo entre  $R$  opciones de equipos de carga, cada una con características técnicas y costos de alquiler diferentes.
  

**Representación del horizonte de tiempo**
  > Considere para la modelación que, se discretiza el día de operación en 37 intervalos de tiempo, de 30 minutos de duración, siendo el primer intervalo de 5:00 a 5:30 y el último intervalo de 23:00 a 23:30.
  
 > **1. Decisiones**:

  > Se desea determinar para cada instante de tiempo $t$, los siguientes aspectos:
>  * Estado del vehículo, para lo cual se considera que el vehiculo puede encontrarse en **solo uno** de los siguientes estados "realizando recorrido $j$", con j=1,...,J, o "en estación de carga".
>  * Potencia de carga de la batería.
>  * Energía almacenada en la batería.
>  * Equipo de carga a arrendar.



> **2. Modelamiento de la operación y requerimientos técnicos**:

> > **2.1 Operación de los colectivos**  <br>
Para modelar la operación de los colectivos debe considerar lo siguiente:
 <br>
  * **Duración del recorrido**: Cada recorrido $j$ requiere de $d_j$ intervalos de tiempo de 30 minutos desde que el vehículo sale de la estación de carga  hasta que llega de vuelta a la estación.  Es decir, una se vez inicia el recorrido, debe mantenerse en recorrido durante $d_j$ instantes de tiempo, y el recorrido termina en el instante de tiempo $t+d_j$. <br><br>
 Para modelar esta condición, le sugiere incluir una variable $Y^{\text{Ini}}_{i,j,t}$ que indique si el vehículo $i$ inicia el recorrido $j$ en el tiempo $t$. Asímismo, se le sugiere incluir una variable $Y^\text{Fin}_{i,j,t}$ que indique si el vehículo $i$ termina el recorrido $j$ en el tiempo $t$.  Así, para modelar el inicio y término del recorrido, debera relacionar la variable que indica si el vehículo $i$ está en el recorrido $j$ en el instante  $t$, $Y_{i,j,t}$, con la variable que indica si el vehículo $i$ está en el recorrido $j$ en el instante anterior, $Y_{i,j,t-1}$, y las variables que indican el inicio y término del recorrido $j$, $Y^\text{Ini}_{i,j,t}$ y $Y^\text{Fin}_{i,j,t}$, respectivamente.  <br><br>
 Para lo anterior, solo existirán cinco combinaciones posibles, las cuales debe considerar para modelar la restricción:
    \begin{array}{|c|c|} \hline
 Y_{i,j,t} & Y_{i,j,t-1} & Y^{\text{Ini}}_{i,j,t} & Y^{\text{Fin}}_{i,j,t} \\ \hline
0 & 0 & 0 & 0 \\
1 & 0 & 1 & 0 \\
0 & 1 & 0 & 1 \\
1 & 1 & 0 & 0 \\
1 & 1 & 1 & 1 \\ \hline
\end{array}
 En base a esto, pude utilizar las variables  $Y^{\text{Ini}}_{i,j,t}$ e $Y^{\text{Fin}}_{i,j,t}$ para representar la condición que si el recorrido inicia en el tiempo $t$, este termina en el instante de tiempo $t+d_j$.
 <br><br> Cabe notar que todos los vehículos se encuentran estacionados en la estación de carga antes de las 5:00. Además, a las 23:00, todos los vehículos deben encontrarse en la estación de carga.
<br><br>
  * **Potencia consumida durante el recorrido** : Cuando el vehículo se encuentra en recorrido $j$, consume una potencia constante de $p_j^{dRec}$ [kW].<br><br>
  * **Límites del estado de batería** : La batería del colectivo eléctrico  $i$ tiene un nivel de almacenamiento máximo de $e_i^\text{Max}$ [kWh], y un nivel de almacenamiento mínimo de $e_i^\text{Min}$ [kWh].
<br><br>
  * **Potencia máxima de carga** : La batería del colectivo eléctrico  $i$ puede cargarse con una potencia que no puede superar $\bar{p}_i^\text{V}$ [k].
  <br><br>
  * **Eficiencia de carga y descarga de la batería**: La carga y descarga de la batería tiene una eficiencia igual a 1.<br><br>
  * **Condición de borde del nivel de almacenamiento**: La batería de los vehículos eléctricos se encuentra completamente cargada a las 5:00.

 > **2.2 Equipo de carga** <br> Como fue mencionado anteriormente, la empresa puede arrendar **solo un** equipo de carga entre $R$ disponibles. Considere que no existe restricción respecto del número de vehículos que pueden cargarse de manera simultanea. No obstante, la operación de estos equipos debe cumplir con el límite de:
 * **Potencia máxima suministrada por del equipo de carga**: La potencia total que suministra el equipo de carga a los vehículos en el instante $t$ no puede superar la potencia máxima $\bar{p}^{\text{EC}}_r$ del cargador $r$ seleccionado.



> **3. Costos e ingresos**:
El objetivo es **maximizar las utilidad diaria**. Para esto considere los  gastos por el arriendo del centro de carga y costos e ingresos de un día completo de operación:

>*   **Costos de arriendo equipo de carga**: Cada equipo de carga $r$ tiene un costo de arriendo por día de $c_r^{\text{EC}} \left[{\$}\right]$.
>*   **Costo de carga de los vehículos**: Cada unidad de energía consumida por el cargador en el instante de tiempo $t$ tiene un costo igual a $c_{t}^{\text{energía}} \left[\frac{\$}{\text{kWh}}\right]$.
>* **Ganancia por recorrido**: por cada recorrido $j$ realizado, se obtiene un ingreso de $g_j^{\text{rec}}$.




# P1. (40 %) Modelamiento del problema de optimización

Modele de forma matemática y generalizada el problema descrito a en la sección anterior. Para ello, incluya:

* Conjuntos de índices.
* Parámetros del problema (datos con los que cuenta).
* Variables de decisión.
* Función objetivo a maximizar
* Restricciones del problema.

###**Indicaciones:**

Escriba el problema de optimización en la celda siguiente.

###**Conjuntos**

  $i$: Conjunto de colectivos eléctricos ($i = 1, \ldots, K$).

  $j$: Conjunto de recorridos disponibles ($j = 1, \ldots, J$).

  $t$: Conjunto de intervalos de tiempo de 30 minutos ($t = 1, \ldots, 37$).

  $r$: Conjunto de opciones de equipos de carga ($r = 1, \ldots, R$).
###**Parámetros del problema**

  $d_j$: Duración del recorrido $j$ en intervalos de tiempo de 30 minutos.

  $p_j^{dRec}$: Potencia consumida durante el recorrido $j$ en kW.

  $e_i^\text{Max}$: Nivel de almacenamiento máximo de la batería del colectivo $i$ en kWh.

  $e_i^\text{Min}$: Nivel de almacenamiento mínimo de la batería del colectivo $i$ en kWh.

  $\bar{p}_i^\text{V}$: Potencia máxima de carga de la batería del colectivo $i$ en kW.

  $\bar{p}^{\text{EC}}_r$: Potencia máxima suministrada por el equipo de carga $r$ en kW.

  $c_r^{\text{EC}}$: Costo de arriendo del equipo de carga $r$ por día en dólares.

  $c_{t}^{\text{energía}}$: Costo de la energía consumida en el intervalo de tiempo $t$ en dólares por kWh.

  $g_j^{\text{rec}}$: Ingreso por realizar el recorrido $j$.

###**Variables de decisión**

  $Y_{i,j,t}$: Variable binaria que indica si el colectivo $i$ está realizando el recorrido $j$ en el intervalo de tiempo $t$.

  $Y^{\text{Ini}}_{i,j,t}$: Variable binaria que indica si el colectivo $i$ inicia el recorrido $j$ en el intervalo de tiempo $t$.

  $Y^{\text{Fin}}_{i,j,t}$: Variable binaria que indica si el colectivo $i$ termina el recorrido $j$ en el intervalo de tiempo $t$.

  $P^{\text{Carga}}_{i,t}$: Potencia de carga de la batería del colectivo $i$ en el intervalo de tiempo $t$ en kW.

  $P^{\text{Descarga}}_{i,t}$: Potencia de descarga de la batería del colectivo $i$ en el intervalo de tiempo $t$ en kW.

  $E^{\text{Bateria}}_{i,t}$: Energía almacenada en la batería del colectivo $i$ en el intervalo de tiempo $t$ en kWh.

  $X_r$: Variable binaria que indica si se selecciona el equipo de carga $r$.

  $S_{i,t}$: Variable binaria que indica si el colectivo $i$ está estacionado en el intervalo de tiempo $t$.

###**Función objetivo**

$$\max{\sum_{j=1}^{J} (g_j^{\text{rec}} \cdot Y^{\text{Ini}}_{i,j,t}) - (\sum_{r=1}^{R} c_r^{\text{EC}} \cdot X_r)} - (\sum_{t=1}^{37} (\sum_{i=1}^{K} P^{\text{Carga}}_{i,t} \cdot c_{t}^{\text{energía}}))
$$
###**Restricciones**
Sujeto a:

Realizando un solo recorrido entre los recorridos disponibles
$$\sum_{j=1}^{J} Y_{i,j,t} \leq 1, \quad \forall i=1,\ldots,K, \forall t=1,\ldots,37$$
Potencia de descarga de la batería del colectivo

$$P^{\text{Descarga}}_{i,t} = \sum_{j=1}^{J} \frac{p_j^{dRec} \cdot Y_{i,j,t}}{\frac{d_j}{2}}, \quad \forall i=1,\ldots,K, \forall t=1,\ldots,37$$

Energía almacenada en la batería del colectivo en cada intervalo de tiempo.
$$E^{\text{Batería}}_{i,t} = E^{\text{Batería}}_{i,t-1} + S_{i,t}*P^{\text{Carga}}_{i,t} - (1-S_{i,t})*P^{\text{Descarga}}_{i,t}, \quad \forall i=1,\ldots,K, \forall t=1,\ldots,37$$
Seleccionar exactamente un equipo de carga
$$\sum_{r=1}^{R} X_r = 1$$
La suma de las potencias de carga de los colectivos eléctricos no sobrepasa la capacidad máxima de carga del equipo seleccionado
$$\sum_{i=1}^{K} P^{\text{Carga}}_{i,t} \leq \bar{p}^{\text{EC}}_r \cdot X_r \cdot S_{i,t}, \quad \forall t=1,\ldots,37, \forall r=1,\ldots,R$$
Nivel de almacenamiento máximo de la batería
$$E^{\text{Batería}}_{i,t} \leq e_i^\text{Max}, \quad \forall i=1,\ldots,K, \forall t=1,\ldots,37$$
Nivel de almacenamiento minimo de la batería
$$E^{\text{Bateria}}_{i,t} \geq e_i^\text{Min}, \quad \forall i=1,\ldots,K, \forall t=1,\ldots,37$$
Un colectivo inicia o finaliza un recorrido y cuándo está realizando un recorrido
$$Y_{i,j,t} - Y_{i,j,t-1} = Y^{\text{Ini}}_{i,j,t} - Y^{\text{Fin}}_{i,j,t}, \quad \forall i=1,\ldots,K, \forall j=1,\ldots,J, \forall t=1,\ldots,37$$
Único intervalo de tiempo en el que el colectivo inicie ese recorrido
$$\sum_{t=1}^{d_j} Y^{\text{Ini}}_{i,j,t} = 1, \quad \forall i=1,\ldots,K, \forall j=1,\ldots,J$$
Único intervalo de tiempo en el que el colectivo finaliza ese recorrido
$$\sum_{t=37-d_j+1}^{37} Y^{\text{Fin}}_{i,j,t} = 1, \quad \forall i=1,\ldots,K, \forall j=1,\ldots,J$$
Vehiculo solo puede estar en 2 estados, en recorrido o estacionado
$$S_{i,t} = 1 - (\sum_{j=1}^{J} Y^{\text{Ini}}_{i,j,t}- \sum_{j=1}^{J} Y^{\text{Fin}}_{i,j,t}), \quad \forall i=1,\ldots,K, \forall t=1,\ldots,37$$

# P2 (40%): Caso de dos vehículos y un solo recorrido

En esta pregunta se aplicará el modelo anterior para estudiar el caso en que la flota de colectivos consiste en dos vehículos eléctricos y que solo existe un recorrido posible. En la primera parte, deberá implementar el modelo usando la librería Pyomo. Luego deberá resolver el modelo, obtener los resultados, y analizarlos.

A continuación se le indicará como cargar los datos para que en la siguiente subsección implemente y resuelva el modelo planteado.

## Carga de datos

A continuación, se cargan los datos para trabajar el caso en que se consideran sólo 2 vehículos y un solo recorrido. Para mejor comprensión de la información con la que cuenta, se le recomienda revisar cada set de datos.

In [ ]:
## Cargar set de datos para trabajar -- P2
## NO MODIFIQUE ESTE CÓDIGO

# Importar datos P2
df_p2 = drive.CreateFile({'id':'1qr3XXuc2S9wJ43cY5pghPwPoDSHp3-1M'})
df_p2.GetContentFile("df_p1.xlsx")
path_data_p2 = '/content/df_p1.xlsx'
print('**** Datos P2 cargados ****')


data_vehiculos = pd.read_excel(path_data_p2, sheet_name = 'vehiculos', index_col = 'id_vehiculo') #Datos de vehiculos
data_recorridos = pd.read_excel(path_data_p2, sheet_name = 'recorridos', index_col = 'id_recorrido') #Datos de recorridos
data_equiposcarga = pd.read_excel(path_data_p2, sheet_name = 'equipos de carga', index_col = 'id_equipo') #Datos de equipos de carga
data_costoenergia = pd.read_excel(path_data_p2, sheet_name = 'precio energía', index_col = 'id_intervalo') #Datos de costo energía

print(data_vehiculos)

**Parámetros vehículos**

Puede acceder a cada parámetro de los **vehiculos** a través de los siguientes vectores ya definidos. Estos parámetros son:

*   Potencia máxima de carga $\overline{p}^{V}_{i}\text{ [kW]}$
*   Energía mínima ${e}^{\text{Min}}_{i}\text{ [kWh]}$
*   Energía máxima ${e}^{\text{Max}}_{i}\text{ [kWh]}$


In [ ]:
## NO MODIFIQUE ESTE CÓDIGO
## Set de vehiculos
vehiculos=list(data_vehiculos.index)
## Parámetros vehiculos
# Potencia máxima
p_max_vehiculo = data_vehiculos['p_max']
# Nivel de almacenamiento máximo
e_max_vehiculo= data_vehiculos['e_max']
# Nivel de almacenamiento mínimo
e_min_vehiculo= data_vehiculos['e_min']

#Así, puede obtener los parámetros de cada vehículo de la siguiente manera
i=1
print("Potencia máxima vehículo",i,":", p_max_vehiculo[i])
print("Energía máxima vehículo",i,":", e_max_vehiculo[i])
print("Energía mínima vehículo",i,":", e_min_vehiculo[i])

**Parámetros equipos de carga**

Puede acceder a cada parámetro de los **equipos de carga** a través de los siguientes vectores ya definidos. Estos parámetros son:

*   Potencia máxima  $\overline{p}^{EC}_{r}\text{ [kW]}$
*   Costo arriendo ${c}^{\text{EC}}_{r} \text{ [\$]}$

In [ ]:
## NO MODIFIQUE ESTE CÓDIGO
## Set de equipos de carga
equiposcarga=list(data_equiposcarga.index)
## Parámetros equipos de carga
# Potencia máxima
p_max_equipo = data_equiposcarga['p_max']
# Costo arriendo
costo_equipo = data_equiposcarga['costo']

**Parámetros recorridos**

Puede acceder a cada parámetro de los **recorridos** a través de los siguientes vectores ya definidos. Estos parámetros son:

*   Potencia consumida  $\overline{p}^{dRec}_{j}\text{ [kW]}$
*   Duración (número de intervalos requeridos para completar viaje ida y vuelta)
*   Ganancia ${g}_{j} \text{ [\$]}$

In [ ]:
## NO MODIFIQUE ESTE CÓDIGO
## Set de recorridos
recorridos=list(data_recorridos.index)
## Parámetros recorridos
# Potencia consumida
p_consumida= data_recorridos['p_drec']
# Ganancia
ganancia = data_recorridos['ganancia']
# Duracion
duracion=data_recorridos['duracion']


**Costo energía**

El costo de la energía para cada instante de tiempo $c^{energía}_{t}\text{ [\$/kWh]}$ se puede obtener del siguiente vector


In [ ]:
## NO MODIFIQUE ESTE CÓDIGO
# costo energía
costo_energia=data_costoenergia['p_energia']

**Duracion de los intervalos**

Para entender mejor la representación del día de operación, puede revisar el inicio y término de cada intervalo de tiempo desde t=1,..,37

In [ ]:
## NO MODIFIQUE ESTE CÓDIGO
# set de intervalos de tiempo
intervalos=list(data_costoenergia.index)
# hora de inicio del intervalo
hora_inicio=data_costoenergia['h_inicio']
# hora de término del intervalo
hora_termino=data_costoenergia['h_fin']
# duración intervalo
deltaT=0.5

#imprimir hora de inicio y término para un cierto t
t=1
print(t,hora_inicio[t],hora_termino[t])
t=37
print(t,hora_inicio[t],hora_termino[t])

## P2 a) Implementación usando Pyomo


  >En esta parte se le pide **implementar el modelo planteado en el punto P1**
  >
  >Para ello, **utilice la librería Pyomo** revisada en clases auxiliares, y de la cual le serán útiles los comandos pyo.Param(), pyo.Constraint(), pyo.Var(), pyo.Set() y pyo.Objective() para escribir los respectivos elementos.
  >
  >Además, el comando .pprint() le permitirá revisar exactamente todos los elementos que va añadiendo a su problema de optimización a medida que los integra en el código, ya sean conjuntos, restricciones, variables, parámetros además de la función objetivo.
  > *   Programe un modelo de optimización concreto o abstracto en Pyomo, **esta decisión queda a su criterio.** Aunque un modelo concreto podría ser más facil e intuitivo de programar.
*  En esta pregunta debe programar las variables del problema, parámetros, función objetivo y restricciones según su modelo matemático.

Sea **ordenad@ con su código** y para mayor orden se le recomienda comentar las funciones y líneas más relevantes.

In [ ]:
# Crear un modelo de Pyomo
model = pyo.ConcreteModel()
init_dict = {(1,1):10}

# Conjuntos
model.Vehiculos = pyo.Set(initialize=vehiculos)  # Conjunto de vehículos
model.Recorridos = pyo.Set(initialize=recorridos)  # Conjunto de recorridos
model.Intervalos = pyo.Set(initialize=intervalos)  # Conjunto de intervalos de tiempo
model.EquiposCarga = pyo.Set(initialize=equiposcarga)  # Conjunto de equipos de carga

# Parámetros
model.PMaxVehiculo = pyo.Param(model.Vehiculos, initialize=p_max_vehiculo)  # Parámetro de potencia máxima de vehículos
model.EMaxVehiculo = pyo.Param(model.Vehiculos, initialize=e_max_vehiculo)  # Parámetro de energía máxima de vehículos
model.EMinVehiculo = pyo.Param(model.Vehiculos, initialize=e_min_vehiculo)  # Parámetro de energía mínima de vehículos
model.PMaxEquipo = pyo.Param(model.EquiposCarga, initialize=p_max_equipo)  # Parámetro de potencia máxima de equipos de carga
model.CostoEquipo = pyo.Param(model.EquiposCarga, initialize=costo_equipo)  # Parámetro de costo de arriendo de equipos de carga
model.PConsumida = pyo.Param(model.Recorridos, initialize=p_consumida)  # Parámetro de potencia consumida en recorridos
model.Ganancia = pyo.Param(model.Recorridos, initialize=ganancia)  # Parámetro de ganancia por recorridos
model.Duracion = pyo.Param(model.Recorridos, initialize=duracion)  # Parámetro de duración de recorridos
model.CostoEnergia = pyo.Param(model.Intervalos, initialize=costo_energia)  # Parámetro de costo de energía en intervalos de tiempo

# Variables de decisión
model.YIni = pyo.Var(model.Vehiculos, model.Recorridos, model.Intervalos, domain=pyo.Binary)  # Variable binaria de inicio de recorrido
model.YFin = pyo.Var(model.Vehiculos, model.Recorridos, model.Intervalos, domain=pyo.Binary)  # Variable binaria de fin de recorrido
model.Y = pyo.Var(model.Vehiculos, model.Recorridos, model.Intervalos, domain=pyo.Binary)  # Variable binaria de estado de recorrido
model.PCarga = pyo.Var(model.Vehiculos, model.Intervalos, domain=pyo.NonNegativeReals)  # Variable de potencia de carga
model.PDescarga = pyo.Var(model.Vehiculos, model.Intervalos, domain=pyo.NonNegativeReals)  # Variable de potencia de descarga
model.EBateria = pyo.Var(model.Vehiculos, model.Intervalos, domain=pyo.NonNegativeReals,initialize=init_dict)  # Variable de nivel de energía en la batería
model.X = pyo.Var(model.EquiposCarga, domain=pyo.Binary)  # Variable binaria para seleccionar equipo de carga
model.S = pyo.Var(model.Vehiculos, model.Intervalos, domain=pyo.Binary)  # Variable binaria de estado de estacionamiento

In [ ]:
# Función objetivo
model.Profit = pyo.Objective(
    expr=sum(model.Ganancia[j] * model.YIni[i, j, t] for i in model.Vehiculos for j in model.Recorridos for t in model.Intervalos)
        - sum(model.CostoEquipo[r] * model.X[r] for r in model.EquiposCarga)
        - sum(model.PCarga[i, t] * model.CostoEnergia[t] for i in model.Vehiculos for t in model.Intervalos),
    sense=pyo.maximize
)

# Restricciones
##Un solo recorrido entre los recorridos disponibles
def recorrido(model, i, t):
    return sum(model.Y[i, j, t] for j in model.Recorridos) <= 1
model.Recorrido = pyo.Constraint(model.Vehiculos, model.Intervalos, rule=recorrido)

# Potencia de descarga de la batería del colectivo
def potencia_descarga(model, i, t):
    return model.PDescarga[i, t] == sum((model.PConsumida[j] * model.Y[i, j, t]) / (model.Duracion[j] / 2) for j in model.Recorridos)
model.PotenciaDescarga = pyo.Constraint(model.Vehiculos, model.Intervalos, rule=potencia_descarga)

# Energía almacenada en la batería
def energia_bateria(model, i, t):
    if t == 1:
        # Añade una restricción para el primer intervalo (t=1)
        return model.EBateria[i, t] == model.EMaxVehiculo[i]  # No hay t-1 para t=1
    else:
        return model.EBateria[i, t] == model.EBateria[i, t-1] + model.PCarga[i, t] * model.S[i, t] - model.PDescarga[i, t] * (1 - model.S[i, t])
model.EnergiaBateria = pyo.Constraint(model.Vehiculos, model.Intervalos, rule=energia_bateria)

#Seleccionar exactamente un equipo de carga
def equipo_carga(model):
    return sum(model.X[r] for r in model.EquiposCarga) == 1
model.EquipoCarga = pyo.Constraint(rule=equipo_carga)

#No sobrepasar la capacidad máxima de carga del equipo
def capacidad_carga(model, r, t):
    return sum(model.PCarga[i, t] for i in model.Vehiculos) <= model.PMaxEquipo[r] * model.X[r]
model.CapacidadCarga = pyo.Constraint(model.EquiposCarga, model.Intervalos, rule=capacidad_carga)

#Almacenamiento máximo de la batería
def max_energia(model, i, t):
    return model.EBateria[i, t] <= model.EMaxVehiculo[i]
model.MaxEnergia = pyo.Constraint(model.Vehiculos, model.Intervalos, rule=max_energia)

#Almacenamiento minimo de la batería
def min_energia(model, i, t):
    return model.EBateria[i, t] >= model.EMinVehiculo[i]
model.MinEnergia = pyo.Constraint(model.Vehiculos, model.Intervalos, rule=min_energia)

#Inicia o finaliza un recorrido
def inicio_fin_recorrido(model, i, j, t):
    if t == list(model.Intervalos)[0]:
        return model.Y[i, j, t] - model.Y[i, j, t] == model.YIni[i, j, t] - model.YFin[i, j, t]
    else:
        return model.Y[i, j, t] - model.Y[i, j, t-1] == model.YIni[i, j, t] - model.YFin[i, j, t]
model.InicioFinRecorrido = pyo.Constraint(model.Vehiculos, model.Recorridos, model.Intervalos, rule=inicio_fin_recorrido)

#Único intervalo de tiempo inicio recorrido
def unico_inicio_recorrido(model, i, j):
    return sum(model.YIni[i, j, t] for t in model.Intervalos) == 1
model.UnicoInicioRecorrido = pyo.Constraint(model.Vehiculos, model.Recorridos, rule=unico_inicio_recorrido)

#Único intervalo de tiempo fin recorrido
def unico_fin_recorrido(model, i, j):
    return sum(model.YFin[i, j, t] for t in model.Intervalos) == 1
model.UnicoFinRecorrido = pyo.Constraint(model.Vehiculos, model.Recorridos, rule=unico_fin_recorrido)

# Restricción para la variable binaria S
def estado_vehiculo(model, i, t):
    return model.S[i, t] == 1 - (sum(model.YIni[i, j, t] for j in model.Recorridos)-sum(model.YFin[i, j, t] for j in model.Recorridos))
model.EstadoVehiculo = pyo.Constraint(model.Vehiculos, model.Intervalos, rule=estado_vehiculo)

model.pprint()

EBateria[1,2]  ==  EBateria[1,1] + PCarga[1,2]*S[1,2] - PDescarga[1,2]*(1 - S[1,2])
EBateria[1,2]  ==  EBateria[1,1] + PCarga[1,2]*S[1,2] - PDescarga[1,2]*(1 - S[1,2])
EBateria[1,3]  ==  EBateria[1,2] + PCarga[1,3]*S[1,3] - PDescarga[1,3]*(1 - S[1,3])
EBateria[1,4]  ==  EBateria[1,3] + PCarga[1,4]*S[1,4] - PDescarga[1,4]*(1 - S[1,4])
EBateria[1,5]  ==  EBateria[1,4] + PCarga[1,5]*S[1,5] - PDescarga[1,5]*(1 - S[1,5])
EBateria[1,6]  ==  EBateria[1,5] + PCarga[1,6]*S[1,6] - PDescarga[1,6]*(1 - S[1,6])
EBateria[1,7]  ==  EBateria[1,6] + PCarga[1,7]*S[1,7] - PDescarga[1,7]*(1 - S[1,7])
EBateria[1,8]  ==  EBateria[1,7] + PCarga[1,8]*S[1,8] - PDescarga[1,8]*(1 - S[1,8])
EBateria[1,9]  ==  EBateria[1,8] + PCarga[1,9]*S[1,9] - PDescarga[1,9]*(1 - S[1,9])
EBateria[1,10]  ==  EBateria[1,9] + PCarga[1,10]*S[1,10] - PDescarga[1,10]*(1 - S[1,10])
EBateria[1,11]  ==  EBateria[1,10] + PCarga[1,11]*S[1,11] - PDescarga[1,11]*(1 - S[1,11])
EBateria[1,12]  ==  EBateria[1,11] + PCarga[1,12]*S[1,12] - PDesc

## P2 b) Resolución del problema

Utilizando el solver cplex, resuelva el problema de optimización. Le será util el código que se le entrega en los bloques a continuación. Si desea usar otro solver, lo puede hacer, pero sea claro y explícito en el desarrollo.

In [ ]:
# Resolución. Se llama al solver y se resuelve el problema.
## NO MODIFIQUE ESTE CÓDIGO
solver = pyo.SolverFactory('cplex', solver_io='python')
filename = 'model.lp'
model.write(filename, io_options={'symbolic_solver_labels': True})
solver.solve(model).write()

CPLEX Error  5002: 'x595' is not convex.


CplexSolverError: ignored

## P2 c) Obtener y gráficar resultados

En esta parte se le pide obtener y analizar los resultados. Para ello siga las instrucciones que se indican a continuación.


**Extraer resultados desde Pyomo**

Los valores de cada variable, tras resolver el problema de optimización, pueden ser obtenidos con el comando .get_values().

Por ejemplo, si su modelo está creado en la variable model y la variable de optimización que desea obtener es model.P, debe utilizar el comando de la siguiente forma:


In [ ]:
resultados_P = model.P.get_values()

Por otro lado, si desea obtener el valor de la función objetivo, y dicha función está definida en model.obj, debe utilizar el comando:


In [ ]:
pyo.value(model.obj)

In [ ]:
# Para poder obtener los resultados en un dataframe indexado, le pude ser util la  funcion crear_df(dict_results, variable_name, index_names),
#la que recibe como entrada
#dict_results: diccionario con los valores óptimos de la variable de interes dict_results
#variable_name: nombre de la variable
#index_names: lista con los nombres de los índices en los cuales está indexada la variable
def crear_df(dict_results, variable_name, index_names):
  var_values = pd.DataFrame.from_dict(dict_results, orient='index', columns=[str(variable_name)])
  var_values.index = pd.MultiIndex.from_tuples(var_values.index)
  var_values.index.names = index_names
  return var_values

#Como ejemplo de uso, supongamos su modelo tiene la variable Y_ini indexada en 'vehiculos', 'recorridos', 'intervalo'.
#Luego, función anterior es aplicada de la siguiente manera:
resultado_Yini=model.Y_ini.get_values() # obtener resultado del problema de optimización
Y_ini_R= crear_df(resultado_Yini, 'Yini', ['vehiculos', 'recorridos', 'intervalo']) #llamar a crear_df
Y_ini_R # mostrar resultado

### Indique:
i. El valor de la función objetivo

ii. El equipo de carga seleccionado

iii. Costo total de la energía consumida

iiii. Cuántas veces se realiza el recorrido $1$ (en este caso sólo existe un recorrido)


Pueden serle útiles las siguientes funciones:

* stackplot() bar() plot() de matplotlib.
* groupby('col').sum() para agrupar por cierta columna 'col'  y obtener la suma de dichos valores




In [ ]:
### Respuesta

### Gráfique:

i. Gráfico apilado de potencia de carga de los vehículos y potencia máxima del equipo de carga en el tiempo

ii. Estado de cada vehículo en el tiempo

iii. Dual de la restricción de balance de energía (vea la guía para obtener los duales) para cada intervalo de tiempo


In [ ]:
### Respuesta



#### Guía para obtener los duales
Para obtener los valores de los duales en pyomo se debe fijar **todas** las variables binarias o enteras como parámetros del problema.
Para fijar los valores de las variables binarias, se usa la función fix().
Así, si  su modelo está creado en el objeto model y sea Yini una variable binaria del modelo, entonces la fijaríamos mediante

In [ ]:
model.Yini.fix()

Luego, debe resolver nuevamente el problema, para ello además debe anteponer una línea de código que le permite calcular los duales. Para el cálculo de duales y además resolver el problema, utilice el siguiente código:



In [ ]:
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
solver = pyo.SolverFactory('cplex', solver_io='python')
solver.solve(model).write()
duals_dict = get_dual(model)



Así, en duals_dict tendrá un diccionario en el que podrá acceder a todos los duales de las restricciones.

Por ejemplo, el modelo contiene la restricción de balance de energía en
model.Balance

Podrá acceder a todos sus valores a través del comando:

In [ ]:
duals_dict['Balance']

# P3 Caso de estudio de mayor tamaño


## Carga de datos

A continuación, se cargan los datos para trabajar el caso con mayor cantidad de vehículos (4) y recorridos (3). Debido a la mayor carga computacional requerida, se trabajará con el solver de Gurobi, el cual para utilizarse requiere de una licencia.

In [ ]:
## Cargar set de datos para trabajar -- P3
## NO MODIFIQUE ESTE CÓDIGO

# Importar datos P3
df_p3 = drive.CreateFile({'id':'1SZc27tFkkG5R5oSDDPqwIJY8ubG_l6P5'})
df_p3.GetContentFile("df_p3.xlsx")
path_data_p3 = '/content/df_p3.xlsx'

# Importar licencia
licencia = drive.CreateFile({'id':'1qmbML1TixikCTRUzuxB4C2-YD1QE0mK6'})
licencia.GetContentFile("gurobi.lic")

print('**** Datos P3 cargados ****')

data_vehiculos = pd.read_excel(path_data_p3, sheet_name = 'vehiculos', index_col = 'id_vehiculo') #Datos de vehiculos
data_recorridos = pd.read_excel(path_data_p3, sheet_name = 'recorridos', index_col = 'id_recorrido') #Datos de recorridos
data_equiposcarga = pd.read_excel(path_data_p3, sheet_name = 'equipos de carga', index_col = 'id_equipo') #Datos de equipos de carga
data_costoenergia = pd.read_excel(path_data_p3, sheet_name = 'precio energía', index_col = 'id_intervalo') #Datos de costo energía

Se definen los parámetros del problema con el nuevo set de datos:

In [ ]:
## NO MODIFIQUE ESTE CÓDIGO
## Set de vehiculos
vehiculos=list(data_vehiculos.index)
## Parámetros vehiculos
# Potencia máxima
p_max_vehiculo = data_vehiculos['p_max']
# Nivel de almacenamiento máximo
e_max_vehiculo= data_vehiculos['e_max']
# Nivel de almacenamiento mínimo
e_min_vehiculo= data_vehiculos['e_min']

In [ ]:
## NO MODIFIQUE ESTE CÓDIGO
## Set de equipos de carga
equiposcarga=list(data_equiposcarga.index)
## Parámetros equipos de carga
# Potencia máxima
p_max_equipo = data_equiposcarga['p_max']
# Costo arriendo
costo_equipo = data_equiposcarga['costo']

In [ ]:
## NO MODIFIQUE ESTE CÓDIGO
## Set de recorridos
recorridos=list(data_recorridos.index)
## Parámetros recorridos
# Potencia consumida
p_consumida= data_recorridos['p_drec']
# Ganancia
ganancia = data_recorridos['ganancia']
# Duracion
duracion=data_recorridos['duracion']

## P3 a) Resolución del problema

Utilizando el mismo modelo de la pregunta 2, resolver el problema con los nuevos datos

In [ ]:
#Escriba aquí el código



## P3 b) Obtener y gráficar resultados

Con los nuevos datos se pide:

### Indique:
i. El valor de la función objetivo

ii. El equipo de carga seleccionado

iii. Costo total de la energía consumida

iiii. Cuántas veces se realiza el recorrido $1$ y $2$

In [ ]:
### Respuesta:



### Gráfique:

i. Gráfico apilado de potencia de carga de los vehículos y potencia máxima del equipo de carga en el tiempo

ii. Estado de cada vehículo en el tiempo

iii.  Dual de la restricción de balance de energía para cada intervalo de tiempo

In [ ]:
### Respuesta:



## P3 c) Preguntas de análisis

Responda las siguientes preguntas en base a los resultados obtenidos para ambas preguntas. Apoyese en los gráficos generados y las características de los sistemas.

1. Analice los horarios de estacionamiento de los vehículos para su carga. ¿Por qué se elige cargar los vehículos en ciertos horarios en lugar de otros momentos del día? Relacione su respuesta con el precio de la energía.

<font color='red'>Respuesta:</font>

2. En relación a la parte 3, ¿qué recorrido se realiza con mayor frecuencia? ¿Cuál es la razón detrás de esta tendencia? Exponga las ventajas de realizar este recorrido en comparación con el otro.

<font color='red'>Respuesta:</font>

3. Describa la elección de los equipos de carga. Fundamente la selección de cada equipo en función de la cantidad de vehículos involucrados.

<font color='red'>Respuesta:</font>

4. Compare el estado (recorrido o carga) de los cuatro vehículos. ¿Existen vehículos que realicen una mayor cantidad de recorridos que otros? De ser ese el caso, ¿por qué cree usted que sucede esta diferencia entre vehículos?

<font color='red'>Respuesta:</font>